I use EXP-4 100 rows for testing mlae 

In [ ]:
import pandas as pd

balanced_df = pd.read_csv('balanceddata.csv')

balanced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2472 entries, 0 to 2471
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   task_name        2472 non-null   int64  
 1   query            2472 non-null   object 
 2   run              2472 non-null   int64  
 3   model_name       2472 non-null   object 
 4   image_path       2472 non-null   object 
 5   ground_truth     2472 non-null   int64  
 6   raw_answer       2472 non-null   object 
 7   time_ms          2472 non-null   float64
 8   parsed_answers   2472 non-null   float64
 9   cleaned_answers  2472 non-null   float64
dtypes: float64(3), int64(3), object(4)
memory usage: 193.2+ KB


First, I would like to know how mlae is being caculated for each model

In [2]:
import pandas as pd
import numpy as np
import sklearn.metrics as metrics

# MLAE Calculation 1: Cleveland-McGill (Log Transformed MAE)
def Cleveland_McGill(y_pred, y_test):
    return np.log2(metrics.mean_absolute_error(y_pred * 100, y_test * 100) + 0.125)

# MLAE Calculation 2: Per-Instance Log Transformed MAE
def MLAE_per_instance(y_pred, y_test):
    mlae_values = [
        np.log2(metrics.mean_absolute_error([gt], [pred]) + 0.125) 
        for gt, pred in zip(y_test, y_pred)
    ]
    return np.mean(mlae_values)  # Average across all instances

# Load dataset
df = balanced_df.copy()

# Compute MLAE for each model
mlae_results = {}

for model, group in df.groupby('model_name'):
    y_test = group['ground_truth'].values
    y_pred = group['cleaned_answers'].values

    mlae_results[model] = (
        Cleveland_McGill(y_pred, y_test),
        MLAE_per_instance(y_pred, y_test)
    )

# Display final MLAE comparison
print("\nMLAE Comparison (Cleveland-McGill vs. Per-Instance MLAE):")
for model, (mlae_Cleveland_McGill, mlae_per_instance) in mlae_results.items():
    print(f"{model}: Cleveland-McGill MLAE = {mlae_Cleveland_McGill:.4f}, Per-Instance MLAE = {mlae_per_instance:.4f}")



MLAE Comparison (Cleveland-McGill vs. Per-Instance MLAE):
CustomLLaMA: Cleveland-McGill MLAE = 8.3361, Per-Instance MLAE = 1.1383
Gemini1_5Flash: Cleveland-McGill MLAE = 7.9064, Per-Instance MLAE = 0.7923
GeminiProVision: Cleveland-McGill MLAE = 8.0186, Per-Instance MLAE = 0.8843
LLaMA: Cleveland-McGill MLAE = 9.1475, Per-Instance MLAE = 1.1656
gpt4o: Cleveland-McGill MLAE = 8.0533, Per-Instance MLAE = 0.8373


My question right now is which one is reasonable for this case?

for finetuned models are okay to use per-instance MLAE caculation, as their predictions are small integers. 

In [3]:
# Get unique predicted values for each model
unique_predictions_per_model = {}

for model, group in df.groupby('model_name'):
    unique_predictions_per_model[model] = group['cleaned_answers'].unique()

# Display unique values for each model
for model, unique_values in unique_predictions_per_model.items():
    print(f"\nUnique Predicted Values for {model}:")
    print(unique_values)


Unique Predicted Values for CustomLLaMA:
[ 9. 10.  3.  2.  7.  4.  1.  6.  5.  8.]

Unique Predicted Values for Gemini1_5Flash:
[7. 6. 8.]

Unique Predicted Values for GeminiProVision:
[7. 2. 4.]

Unique Predicted Values for LLaMA:
[  4.   5.   3.  10.   6.  15.   7. 100.   2.   8.   9.  20.  52.  36.
  50.   1. 145.  35.  40.  30.  13. 140.  24. 152.  55.  90.  45. 300.
  16.]

Unique Predicted Values for gpt4o:
[ 9. 10.  5.  7.  3.  8.  6.  4.  2.]


I see unique predicted value for llama model: they have larger value.

Caculate gpt-4o's mlae

In [12]:
# Filter data for GPT-4o
gpt4o_df = df[df['model_name'] == 'gpt4o']

# List unique predicted values
unique_predictions = gpt4o_df['cleaned_answers'].unique()
print("Unique Predicted Values for GPT-4o:")
print(unique_predictions)

# Example Calculation for GPT-4o
if not gpt4o_df.empty:
    example_y_test = gpt4o_df['ground_truth'].values[:1000]
    example_y_pred = gpt4o_df['cleaned_answers'].values[:1000]

    mlae_Cleveland_McGill = Cleveland_McGill(example_y_pred, example_y_test)
    mlae_per_instance = MLAE_per_instance(example_y_pred, example_y_test)

    print("\nExample MLAE Calculation for GPT-4o (First 5 Rows):")
    print(f"Ground Truth: {example_y_test}")
    print(f"Predictions: {example_y_pred}")
    print(f"Cleveland-McGill MLAE: {mlae_Cleveland_McGill:.4f}")
    print(f"Per-Instance MLAE: {mlae_per_instance:.4f}")
else:
    print("\nNo data found for GPT-4o.")


Unique Predicted Values for GPT-4o:
[ 9. 10.  5.  7.  3.  8.  6.  4.  2.]

Example MLAE Calculation for GPT-4o (First 5 Rows):
Ground Truth: [ 7 10  9  9  6  3  2  1  2  8  1  5  1  9  4  3  7 10  3  9  6  4  8  6
  2  8  9 10  8  1  8  5  9  3  7  7  7  7  8  6  3  5  7  2  5  7  4  3
  8 10  1  5  7  9  6  9  8  9  3  8  1 10 10  2 10  6  5  5  5  4  5  8
  2  5  2  3  1  1  6 10  6  9  4  7  9 10  8  8  6  8  7 10  5  3 10  6
  6  5  7  6  2  6 10 10  6 10  8  4 10  5  3  4  9  6 10  9  4  8  5  3
  4  7  7  8  2  4  8  9  4  9  3  6  5  7  3  3  3  8  8  5  7  3  3  7
  3  5  9  4 10  7  8  6  3  4  9  9 10  6  4  5  8  3  3  1  4  2  1  1
  7  6  6  9  9  8  3  1  9 10  2  4  1  4  3  1  4  7  7  2  3  1  8  8
  1  7  5  8  5 10  8  6  9  8  7 10  1  6  6  6  4  9  4  8  1  7  1  8
  1  9  7  8  8  6  7  8  3  4  6  9  7  6  7  4  3  3  7  8  3  7  6  3
  2  1  8  6  1  9  3  3  9  1  5  9  2  6  6  4  8  7  6  2  8  1  2  4
  1 10  1  6  9  8  8  2  9  5  3  1  6  9  6  6  3  9  

Llama's mlae caculation

In [8]:
# Filter data for LLaMA
llama_df = df[df['model_name'] == 'LLaMA']

# List unique predicted values
unique_predictions_llama = llama_df['cleaned_answers'].unique()
print("Unique Predicted Values for LLaMA:")
print(unique_predictions_llama)

# Example Calculation for LLaMA
if not llama_df.empty:
    example_y_test = llama_df['ground_truth'].values[:1000]
    example_y_pred = llama_df['cleaned_answers'].values[:1000]

    mlae_Cleveland_McGill = Cleveland_McGill(example_y_pred, example_y_test)
    mlae_per_instance = MLAE_per_instance(example_y_pred, example_y_test)

    print("\nExample MLAE Calculation for LLaMA (First 5 Rows):")
    #print(f"Ground Truth: {example_y_test}")
    print(f"Predictions: {example_y_pred}")
    print(f"Cleveland-McGill MLAE: {mlae_Cleveland_McGill:.4f}")
    print(f"Per-Instance MLAE: {mlae_per_instance:.4f}")
else:
    print("\nNo data found for LLaMA.")


Unique Predicted Values for LLaMA:
[  4.   5.   3.  10.   6.  15.   7. 100.   2.   8.   9.  20.  52.  36.
  50.   1. 145.  35.  40.  30.  13. 140.  24. 152.  55.  90.  45. 300.
  16.]

Example MLAE Calculation for LLaMA (First 5 Rows):
Predictions: [  4.   5.   5.   3.  10.   6.   4.  15.   6.   6.   5.   7.   6.   6.
  10.   4.   5.   6.   6.   7.   5.   5.   6.   6.  10.   7.   4.   5.
 100.   4.   2.   6.   4.   6.   6.   3.   4.   4.   6.   4.   6.   4.
   3.   7.   7.   7.   4.   4.   7.   7.  10.   7.   6.   6.   5.   7.
   7.   5.   6.   3. 100.  10.   8.   6.   6.   4.   7.   7.   5.   9.
   7.   6.   6.   4.   4.  20.   6.   6.   3.   3.   3.   5.   5.  52.
  36.   4.   3.   5.   6.   6.   6.   3.   6.   6.   4.  10.   5.  50.
   4.   6.  10.   6.   5.   7.   5.   6.   6.   7.   5.   3.   3.   6.
   5.   3.   7.   4.   2.   5.   3.   5.  10.   6.   5.   5.   4.   6.
   7.   7.   1.  10.   5.   6.   6.   3.   5.   2.   3.   4.   5.   3.
   5.   6.   6.   3.   4.   4.   6.   6. 

In [14]:
# Filter data for LLaMA
llama_df = df[df['model_name'] == 'CustomLLaMA']

# List unique predicted values
unique_predictions_llama = llama_df['cleaned_answers'].unique()
print("Unique Predicted Values for custom LLaMA:")
print(unique_predictions_llama)

# Example Calculation for LLaMA
if not llama_df.empty:
    example_y_test = llama_df['ground_truth'].values[:1000]
    example_y_pred = llama_df['cleaned_answers'].values[:1000]

    mlae_Cleveland_McGill = Cleveland_McGill(example_y_pred, example_y_test)
    mlae_per_instance = MLAE_per_instance(example_y_pred, example_y_test)

    print("\nExample MLAE Calculation for LLaMA (First 5 Rows):")
    #print(f"Ground Truth: {example_y_test}")
    print(f"Predictions: {example_y_pred}")
    print(f"Cleveland-McGill MLAE: {mlae_Cleveland_McGill:.4f}")
    print(f"Per-Instance MLAE: {mlae_per_instance:.4f}")
else:
    print("\nNo data found for LLaMA.")


Unique Predicted Values for custom LLaMA:
[ 9. 10.  3.  2.  7.  4.  1.  6.  5.  8.]

Example MLAE Calculation for LLaMA (First 5 Rows):
Predictions: [ 9. 10.  3.  3. 10.  3.  2.  7.  4.  1.  2.  9.  9.  6.  1.  4.  4.  9.
  6.  5.  1. 10. 10.  2.  2.  5.  1.  3.  6.  4.  9.  1.  6.  8.  1.  3.
  3.  5. 10.  1.  7.  7.  4.  3.  2. 10.  6.  3. 10.  5.  5.  6.  9.  7.
  3.  4.  1.  3.  2.  3.  5. 10.  6.  6.  2.  4.  6. 10.  5. 10.  1.  7.
  9. 10.  9.  3.  5.  6.  3.  5.  3. 10. 10.  9.  6.  1.  4.  5.  6.  3.
  3.  6.  1.  9.  3.  8.  4.  7. 10.  8.  8.  5.  5.  6.  7.  5.  4.  2.
  9.  5.  7. 10.  1.  5.  5.  8.  2.  7. 10.  4.  8.  2. 10. 10. 10.  1.
  5.  3.  6. 10.  9.  2.  2.  9.  7.  1.  9.  9.  9.  1.  3.  3.  1.  9.
  2.  6.  9.  2.  3.  2.  1.  6.  4.  9.  1.  9.  5.  7.  8.  3.  7.  2.
  7.  5.  2.  5. 10.  8. 10.  9.  7.  9.  6.  5.  1.  7.  6.  3.  2.  1.
  6.  9.  3.  4.  9.  4.  5.  9.  4.  2.  3.  7.  6.  6.  9. 10. 10. 10.
  6.  2.  1.  3.  1.  8. 10.  1.  2.  9.  9.  9.